In [2]:
import math
import numpy as np

## Mechanical properties of objects/shapes

In [3]:
# Pure shape construction class:
class pure_shape(object):
    def __init__(self, shape, kwargs, pos_shape=1):
        self.pos_shape = pos_shape
        self.shape = shape
        if shape == "rectangle":
            self.b = kwargs['b']
            self.h = kwargs['h']
            self.coords = kwargs['coords']
        self.centroid = pure_shape.computeCentroid(self)
        self.areaMoments = pure_shape.computeAreaMoments(self)
        self.area = pure_shape.computeArea(self)
        self.areaProduct = pure_shape.computeAreaProduct(self)
        self.principleAngle = pure_shape.computePrincipleAngle(self)
        
    def getCentroid(self):
        return self.centroid
    
    def computeCentroid(self):
        if self.shape == "rectangle":
            x = self.coords[0][0] + 0.5*self.b
            y = self.coords[0][1] + 0.5*self.h
        return (x, y)
    
    def getArea(self):
        return self.area
    
    def computeArea(self):
        if self.shape == "rectangle":
            area = self.b*self.h
        return area*self.pos_shape
    
    def getAreaMoments(self):
        return self.areaMoments
    
    def computeAreaMoments(self):
        areaMoments = {'Ix': 0, 'Iy': 0}
        if self.shape == "rectangle":
            areaMoments['Ix'] = self.b*(self.h)**3/12
            areaMoments['Iy'] = self.h*(self.b)**3/12
        
        areaMoments['Jo'] = areaMoments['Ix'] + areaMoments['Iy']
        return areaMoments
    
    def getAreaProduct(self):
        return self.areaProduct
    
    def computeAreaProduct(self):
        if self.shape == "rectangle":
            return 0
    
    def getParams(self):
        params = {}
        if self.shape == "rectangle":
            params['b'] = self.b
            params['h'] = self.h
        return params

    def parallelAxisThmForAreaMoment(self, shift_coords):
        y_dist = abs(self.centroid[1]-shift_coords[1])
        x_dist = abs(self.centroid[0]-shift_coords[0])
        shift_x = self.areaMoments['Ix'] + self.area*(y_dist)**2
        shift_y = self.areaMoments['Iy'] + self.area*(x_dist)**2
        shift_o = self.areaMoments['Jo'] + self.area*(pure_shape.pythag(x_dist, y_dist))**2
        return {'Ix': shift_x, 'Iy': shift_y, 'Jo': shift_o}
    
    def parallelAxisThmForAreaProduct(self, shift_coords):
        y_delta = shift_coords[1]-self.centroid[1]
        x_delta = shift_coords[0]-self.centroid[0]
        return self.areaProduct + self.area*x_delta*y_delta
    
    def inclinedAxis(self, theta):
        Ix = self.areaMoments['Ix']
        Iy = self.areaMoments['Iy']
        Ixy = self.areaProduct
        
        newMomentsAndProd = {}
        ave = (Ix + Iy)/2
        ave_diff = (Ix - Iy)/2
        g = ave_diff*math.cos(2*theta)
        h = Ixy*math.sin(2*theta)
        
        newMomentsAndProd['Ix'] = ave + g - h
        newMomentsAndProd['Iy'] = ave - g + h
        if abs(theta - self.principleAngle) < 1e-5:
            newMomentsAndProd['Ixy'] = 0
        else:
            newMomentsAndProd['Ixy'] = ave_diff*math.sin(2*theta) + Ixy*math.cos(2*theta)
        
        return newMomentsAndProd
    
    def getPrincipleAngle(self):
        return self.principleAngle
    
    def computePrincipleAngle(self):
        areaMoments = self.areaMoments
        if areaMoments['Ix'] == areaMoments['Iy']:
            return 0
        else:
            return 0.5*math.atan(-self.areaProduct*2/(areaMoments['Ix']-areaMoments['Iy']))
    
    # Functional methods:
    
    def pythag(*args):
        sum = 0
        for el in args:
            sum += el**2
        return math.sqrt(sum)
    
    def __str__(self):
        output = f"{self.shape} at {self.coords} with "
        if self.shape == "rectangle":
            output += f"b = {self.b}, h = {self.h}"
        return output + f" area = {self.area}"



# Composite shape construction class:
class composite_shape(pure_shape):
    def __init__(self, *args):
        self.component_shapes = args
        self.centroid = composite_shape.computeCentroid(self, *args)
        self.area = composite_shape.computeArea(self, *args)
        self.areaMoments = composite_shape.computeAreaMoments(self, *args)
        self.areaProduct = composite_shape.computeAreaProduct(self, *args)
        self.principleAngle = super().computePrincipleAngle()
        
    def computeCentroid(self, *args):
        xsum, ysum, asum = 0, 0, 0
        for shape in args:
            a = shape.getArea()
            x, y = shape.getCentroid()
            xsum += a*x
            ysum += a*y
            asum += a
        return (xsum/asum, ysum/asum)
    
    def computeArea(self, *args):
        area = 0
        for shape in args:
            area += shape.getArea()
        return area
    
    def computeAreaMoments(self, *args):
        Ix, Iy = 0, 0
        shift_coords = self.centroid
        for shape in args:
            shifted_shape = shape.parallelAxisThmForAreaMoment(shift_coords)
            Ix += shifted_shape['Ix']
            Iy += shifted_shape['Iy']
        areaMoments = {}
        areaMoments['Ix'] = Ix
        areaMoments['Iy'] = Iy
        areaMoments['Jo'] = Ix + Iy
        return areaMoments
    
    def computeAreaProduct(self, *args):
        Ixy = 0
        shift_coords = self.centroid
        for shape in args:
            Ixy += shape.parallelAxisThmForAreaProduct(shift_coords)
        return Ixy
    
    
    def getComponentShapes(self):
        return self.component_shapes

In [4]:
kwargs = {'b': 20,
          'h': 100,
          'coords': ((0,0), (20,100))}
rect1 = pure_shape("rectangle", kwargs)
print(rect1)
print(rect1.getParams())
print(rect1.computeCentroid())
print(rect1.getCentroid())
print("")
print(rect1.getAreaMoments())
print(rect1.getArea())
print("")
print(rect1.parallelAxisThmForAreaMoment((25, 25)))
print(rect1.getAreaProduct())
print("")
print(rect1.parallelAxisThmForAreaProduct((25, 25)))

rectangle at ((0, 0), (20, 100)) with b = 20, h = 100 area = 2000
{'b': 20, 'h': 100}
(10.0, 50.0)
(10.0, 50.0)

{'Ix': 1666666.6666666667, 'Iy': 66666.66666666667, 'Jo': 1733333.3333333335}
2000

{'Ix': 2916666.666666667, 'Iy': 516666.6666666667, 'Jo': 3433333.3333333335}
0

-750000.0


In [5]:
kwargs1 = {'b': 20,
          'h': 100,
          'coords': ((0,0), (20,100))}
rect1 = pure_shape("rectangle", kwargs1)
print(rect1)

kwargs2 = {'b': 60,
          'h': 20,
          'coords': ((20,80), (80,100))}
rect2 = pure_shape("rectangle", kwargs2)
print(rect2)

rectangle at ((0, 0), (20, 100)) with b = 20, h = 100 area = 2000
rectangle at ((20, 80), (80, 100)) with b = 60, h = 20 area = 1200


In [6]:
comp1 = composite_shape(rect1, rect2)
print(comp1.getCentroid())
print(comp1.getArea())
print(comp1.getAreaMoments())

(25.0, 65.0)
3200
{'Ix': 2906666.666666667, 'Iy': 1626666.6666666667, 'Jo': 4533333.333333334}


In [7]:
areaMoments = comp1.getAreaMoments()
Ix = round(areaMoments['Ix']/1e6, 3)
Iy = round(areaMoments['Iy']/1e6, 3)
print(f"Ix = {Ix}e6 mm^4, Iy = {Iy}e6 mm^4")

Ix = 2.907e6 mm^4, Iy = 1.627e6 mm^4


In [8]:
Ixy = comp1.getAreaProduct()
print(f"Ixy = {round(Ixy/1e6, 3)}e6 mm^4")

Ixy = 1.2e6 mm^4


In [9]:
# Q1 Tut 0
kw1 = {'b': 20,'h': 100,'coords': ((0,0), (20,100))}
rect1 = pure_shape("rectangle", kw1)
kw2 = {'b': 60,'h': 20,'coords': ((20,80), (80,100))}
rect2 = pure_shape("rectangle", kw2)

compQ1 = composite_shape(rect1, rect2)
theta_p = compQ1.getPrincipleAngle()
print(f"θp = {theta_p}")
print(compQ1.inclinedAxis(theta_p))

θp = -0.540419500270584
{'Ix': 3626666.666666667, 'Iy': 906666.666666667, 'Ixy': 0}


In [10]:
# Q2 Tut 0
kw1 = {'b': 120, 'h': 10, 'coords': ((0, 0), (120, 10))}
kw2 = {'b': 10, 'h': 40, 'coords': ((0, 10), (10, 50))}
kw3 = {'b': 10, 'h': 40, 'coords': ((110, -40), (120, 0))}
kw = (kw1, kw2, kw3)
shapes = []
for k in kw:
    shapes.append(pure_shape("rectangle", k))

comp = composite_shape(*shapes)
areaMoments = comp.getAreaMoments()
print(areaMoments)
print(comp.getAreaProduct()/1e6)

{'Ix': 616666.6666666666, 'Iy': 3866666.666666666, 'Jo': 4483333.333333333}
-1.1


In [11]:
# Q3 Tut 0
kw1 = {'b': 200, 'h': 40, 'coords': ((40, 80), (240, 120))}
kw2 = {'b': 40, 'h': 200, 'coords': ((0, 0), (40, 200))}
#kw3 = {'b': 80, 'h': 40, 'coords': ((40, 120), (0, 40))}
kw = (kw1, kw2)#, kw3)
shapes = []
for k in kw:
    shapes.append(pure_shape("rectangle", k))
    aM = shapes[-1].getAreaMoments()
    print(shapes[-1].getCentroid(), aM['Ix']/1e6, aM['Iy']/1e6)

print()

shift_coords = (20, 100)
comp = composite_shape(*shapes)
print(f"centroid @ {comp.getCentroid()}")
areaMoments = comp.parallelAxisThmForAreaMoment(shift_coords)
print(areaMoments)
print(comp.getAreaProduct()/1e6)

print(0.5*(areaMoments['Ix'] + areaMoments['Iy'])/1e6)

new_coords = (0, 100)
print(f"\nareaProd@{new_coords} = {comp.parallelAxisThmForAreaProduct(new_coords)}")

(140.0, 100.0) 1.0666666666666667 26.666666666666668
(20.0, 100.0) 26.666666666666668 1.0666666666666667

centroid @ (80.0, 100.0)
{'Ix': 27733333.333333336, 'Iy': 142933333.33333334, 'Jo': 170666666.6666667}
0.0
85.33333333333334

areaProd@(0, 100) = 0.0


In [12]:
#Q4 Tut 0
kw1 = {'b': 120, 'h': 40, 'coords': ((40, 160), (160, 200))}
kw2 = {'b': 40, 'h': 200, 'coords': ((0, 0), (40, 200))}
kw3 = {'b': 80, 'h': 40, 'coords': ((40, 0), (120, 40))}
kw = (kw1, kw2, kw3)

shapes = []
for k in kw:
    shapes.append(pure_shape("rectangle", k))

comp = composite_shape(*shapes)
print(comp.getAreaMoments())
print("Ixy =", comp.getAreaProduct()/1e6)
theta_p = comp.getPrincipleAngle()
print(f"θp = {theta_p} rad ({round(theta_p*180/math.pi, 2)} deg)")
areaMoments = comp.inclinedAxis(theta_p)
print(f"Ix = {areaMoments['Ix']/1e6}, Iy = {areaMoments['Iy']/1e6}")

{'Ix': 77909333.33333334, 'Iy': 30037333.333333332, 'Jo': 107946666.66666667}
Ixy = 10.752
θp = -0.21109333322274648 rad (-12.09 deg)
Ix = 80.21333333333334, Iy = 27.73333333333333


In [13]:
A = np.array([[1, 1, 1],
              [0, 0.4, 1.2],
              [0.8, -1.2, 0.4]])
B = np.array([[230],
              [184],
              [0]])
X = np.linalg.inv(A)@B

In [14]:
print(X)

[[ 32.85714286]
 [ 65.71428571]
 [131.42857143]]


In [15]:
np.linalg.det(A)

2.2399999999999998

In [16]:
area = math.pi*(12.5)**2
for el in X:
    print(el[0]/area*1e9/1e6)

66.93602178036278
133.8720435607257
267.7440871214514


In [17]:
# 6-109
kw1 = {'b': 25, 'h': 300, 'coords': ((0, 0), (25, 300))}
kw2 = {'b': 25, 'h': 300, 'coords': ((125, 0), (150, 300))}
kw3 = {'b': 100, 'h': 50, 'coords': ((25, 0), (125, 50))}
kw4 = {'b': 100, 'h': 50, 'coords': ((25, 150+100), (125, 300))}
kw = (kw1, kw2, kw3, kw4)

shapes = []
for k in kw:
    shapes.append(pure_shape("rectangle", k))
comp = composite_shape(*shapes)
print(comp.getAreaMoments())
print(comp.getCentroid())

print(comp.getPrincipleAngle())

def d_dx(f, x, h):
    return (f(x+h) - f(x-h))/2/h

Ix = comp.getAreaMoments()['Ix']
Iy = comp.getAreaMoments()['Iy']
f = lambda th: 2*(2)**0.5*math.cos(th)/Ix - 2*(2)**0.5*math.sin(th)/Iy
xi = 0.1 # rad
prev_ans = 1000
while abs(prev_ans - xi) > 1e-6:
    prev_ans = xi
    xi = xi - f(xi)/d_dx(f, xi, 1e-6)
    
print(f"xi = {xi} rad ({xi*180/(math.pi)} deg)")

{'Ix': 270833333.3333334, 'Iy': 67708333.33333333, 'Jo': 338541666.6666667}
(75.0, 150.0)
-0.0
xi = 0.24497866312686412 rad (14.036243467926477 deg)


In [18]:
round((2*(2)**0.5*150/Ix*10**12 + 2*(2)**0.5*75/Iy*10**12)/1e6, 4)

4.6995

In [19]:
f(76/180*math.pi)

-3.8006342966079576e-08

In [20]:
76/180*math.pi

1.3264502315156905

In [21]:
#
kw1 = {'b': 0.174, 'h': 0.012, 'coords': ((0, 0), (0.174, 0.012))}
kw2 = {'b': 0.274, 'h': 0.012, 'coords': ((-0.05, 0.15), (0.224, 0.174))}
kw3 = {'b': 0.012, 'h': 0.138, 'coords': ((0, 0.012), (0.012, 0.15))}
kw4 = {'b': 0.012, 'h': 0.138, 'coords': ((0+0.162, 0.012), (0.012+0.162, 0.15))}
kw = (kw1, kw2, kw3, kw4)

shapes = []
for k in kw:
    shapes.append(pure_shape("rectangle", k))
comp = composite_shape(*shapes)
print(comp.getAreaMoments())
print(comp.getCentroid())

{'Ix': 3.462833555801105e-05, 'Iy': 4.760862400000001e-05, 'Jo': 8.223695955801106e-05}
(0.087, 0.09135911602209944)


## Next part of course

In [22]:
T = np.array([[5, 0, 0],
              [0, -6, -12],
              [0, -12, 1]])
w, v = np.linalg.eig(T) # w = eigenvalues, v = eigenvectors

In [23]:
print(w)

[ 10. -15.   5.]


In [194]:
class stress_tensor(object):
    def __init__(self, T):
        shape = T.shape
        assert shape[0] == shape[1] and len(shape) == 2, "Array must be 2D or 3D, and must be square"
        self.dim = shape[0]
        self.T = T
        
        self.principle_stress = self.__class__.computePrincipleStress(self)
        self.max_shear = self.__class__.computeMaxShear(self)
        self.max_inplane_shear = self.__class__.computeMaxInplaneShear(self)
    
#         if self.dim == 2:
#             self.max_inplane_shear = self.__class__.computeMaxInplaneShear(self)
#         elif self.dim == 3:
#             self.max_inplane_shear = self.max_shear
    
    def getT(self):
        return self.T
    
    def getPrincipleStress(self):
        return self.principle_stress
    
    def computePrincipleStress(self):
#         if self.dim == 2:
#             a = (self.T[0][0] + self.T[1][1])/2
#             b = self.__class__.computeMaxInplaneShear(self)
#             L = [a+b, a-b]
#             L.sort()
#             L.reverse()
#             principle_stress = np.zeros((2, 2))
#             principle_stress[0][0] = L[0]
#             principle_stress[1][1] = L[1]
#             print(f"Fuck {principle_stress}")
#             return principle_stress
        
        w, v = np.linalg.eig(self.T)
        w.sort()
        w = np.flip(w)
        principle_stress = np.zeros(self.T.shape)
        
        for i in range(0, self.T.shape[0]):
            principle_stress[i][i] = round(w[i], 12)
        return principle_stress
    
    def getMaxShear(self):
        return self.max_shear
    
    def computeMaxShear(self):
        p_s = self.principle_stress
        if self.dim == 2:
            p_s = np.c_[p_s, np.zeros((2,1))]
            p_s = np.vstack([p_s, np.zeros(3)])
            # print(p_s)
        
        L = []
        for i in range(3):
            if i == 2:
                L.append(abs(p_s[0][0] - p_s[-1][-1])/2)
            else:
                L.append(abs(p_s[i][i] - p_s[i+1][i+1])/2)
                # print(L)
        
        return max(L)
    
    def getMaxInplaneShear(self):
        return self.max_inplane_shear
    
    def computeMaxInplaneShear(self):
        # print(f"self.T[0][0] = {self.T[0][0]}\nself.T[1][1] = {self.T[1][1]}\nself.T[0][1] = {self.T[0][1]}")
        a = (((self.T[0][0] - self.T[1][1])/2)**2 + self.T[0][1]**2)**0.5
        return a

In [200]:
tens_T = stress_tensor(T)
print(tens_T.getPrincipleStress())
print(tens_T.getMaxShear())
print(tens_T.getMaxInplaneShear())

[[ 10.   0.   0.]
 [  0.   5.   0.]
 [  0.   0. -15.]]
12.5
5.5


In [196]:
tens_G = stress_tensor(np.array([[4, 4.3],[4.3, 2.3]]))
print(tens_G.getPrincipleStress())
print(tens_G.getMaxShear())
print(tens_G.getMaxInplaneShear())

[[ 7.53320659  0.        ]
 [ 0.         -1.23320659]]
4.383206588789
4.383206588788624


In [197]:
f = np.array([[9, 0],
              [0, 4]])
print(f)
g = np.c_[f, [[1], [2]]]
print(g)
h = np.vstack([f, [[1, 2]]])
print(h)

[[9 0]
 [0 4]]
[[9 0 1]
 [0 4 2]]
[[9 0]
 [0 4]
 [1 2]]


In [198]:
H = np.array([[50, 120, -40],
              [120, -100, 0],
              [-40, 0, 70]])
tens_H = stress_tensor(H)
print(tens_H.getPrincipleStress())
print(tens_H.getMaxShear())
print(tens_H.getMaxInplaneShear())

[[ 135.54532182    0.            0.        ]
 [   0.           52.57249085    0.        ]
 [   0.            0.         -168.11781267]]
151.831567247144
141.50971698084905


In [199]:
I = np.array([[10, 2],
              [2, 10]])

tens_I = stress_tensor(I)
print(tens_I.getPrincipleStress())
print(tens_I.getMaxInplaneShear())
print(tens_I.getMaxShear())

[[12.  0.]
 [ 0.  8.]]
2.0
6.0
